In [ ]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, GlobalAveragePooling2D, AveragePooling2D, Activation
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
img_rows, img_cols = 28, 28
num_classes = 10

def data_prep(raw):
    out_y = keras.utils.to_categorical(raw.label, num_classes)

    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    out_x = x_shaped_array / 255
    return out_x, out_y

datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

model = Sequential()
model.add(Conv2D(4, kernel_size=(3, 3),activation='relu', input_shape=(img_rows, img_cols, 1)))
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3),activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3),activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
model.summary()

train_file = "../input/digit-recognizer/train.csv"
raw_data = pd.read_csv(train_file)
x, y = data_prep(raw_data)

X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size = 0.1)
print(X_train)
print(X_val)
print(Y_train)
print(Y_val)
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=64),
        epochs = 20, 
        validation_data = (X_val,Y_val))






In [ ]:
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
test = pd.read_csv("../input/digit-recognizer/test.csv")
test_x = test.values.astype('float32')
test_x = test_x/255.0
test_x = test_x.reshape(test_x.shape[0], 28, 28,1)
results = model.predict_classes(test_x)

In [ ]:
my_submission=pd.DataFrame({"ImageId": list(range(1,len(results)+1)),
                         "Label": results})
my_submission.to_csv('submission.csv', index=False)